In [ ]:
#using calysto bash kernel

In [4]:
conda activate scan-03-filter_genotypes

(scan-03-filter_genotypes) 


In [5]:
mkdir -p /master/nplatt/sch_hae_scan/results/post_phase_filtering && cd /master/nplatt/sch_hae_scan/results/post_phase_filtering

(scan-03-filter_genotypes) 


# Remove "related" samples

In [3]:
#calculate relatedness and then remove any below a cutoff of 0.025 (https://www.nature.com/articles/ng.608)
vcftools \
    --vcf ../phasing/beagle.vcf \
    --not-chr NC_067195.1 \
    --recode \
    --recode-INFO-all \
    --stdout \
    >autosomes.vcf
#After filtering, kept 24_638_821 out of a possible 31_866_337 Sites

(scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) 


In [6]:
plink2 \
    --vcf autosomes.vcf \
    --make-bed \
    --allow-extra-chr \
    --double-id \
    --out autosomes

PLINK v2.00a3.3LM 64-bit Intel (3 Jun 2022)    www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to autosomes.log.
Options in effect:
  --allow-extra-chr
  --double-id
  --make-bed
  --out autosomes
  --vcf autosomes.vcf

Start time: Wed Apr 19 15:45:32 2023
1031287 MiB RAM detected; reserving 515643 MiB for main workspace.
Using up to 192 threads (change this with --threads).
--vcf: 28590241 variants scanned.
--vcf: autosomes-temporary.pgen + autosomes-temporary.pvar.zst +
autosomes-temporary.psam written.
171 samples (0 females, 0 males, 171 ambiguous; 171 founders) loaded from
autosomes-temporary.psam.
28590241 variants loaded from autosomes-temporary.pvar.zst.
Note: No phenotype data present.
Writing autosomes.fam ... done.
Writing autosomes.bim ... done.
Writing autosomes.bed ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767

In [7]:
#add number to front of position to store as "autosome"
sed -i 's/^\(NC_067..\)\([0-9]\)\(.1*\)/\2:\1\2\3/g' autosomes.bim

awk 'BEGIN {FS=":"}; {print $1+1":"$2":"$3}' autosomes.bim >tmp.bim
sort -k1,1 -k4,4n tmp.bim >sorted.bim
mv sorted.bim autosomes.bim

rm tmp.bim

(scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) (scan-03-filter_genotypes) 


In [8]:
king \
    -b autosomes.bed \
    --unrelated

KING 2.2.7 - (c) 2010-2021 Wei-Min Chen

The following parameters are in effect:
                   Binary File :   autosomes.bed (-bname)

Additional Options
         Close Relative Inference : --related, --duplicate
   Pairwise Relatedness Inference : --kinship, --ibdseg, --ibs, --homog
              Inference Parameter : --degree, --seglength
         Relationship Application : --unrelated [ON], --cluster, --build
                        QC Report : --bysample, --bySNP, --roh, --autoQC
                     QC Parameter : --callrateN, --callrateM
             Population Structure : --pca, --mds
              Structure Parameter : --projection, --pcs
              Disease Association : --tdt
   Quantitative Trait Association : --mtscore
                Association Model : --trait [], --covariate []
            Association Parameter : --invnorm, --maxP
               Genetic Risk Score : --risk, --model [], --prevalence, --noflip
              Computing Parameter : --cpus
             

In [9]:
cut -f1 kingunrelated.txt >relatives_to_keep.txt

(scan-03-filter_genotypes) 


In [10]:
cut -f1 kingunrelated_toberemoved.txt

bovis_tanzania_SRR7867225
bovis_tanzania_SRR7867226
ssp_niger_libore_173
ssp_cdivoire_allokokro_104
sha_swaz_unk_187
(scan-03-filter_genotypes) 


In [11]:
vcftools \
    --vcf ../phasing/beagle.vcf \
    --keep relatives_to_keep.txt   \
    --recode \
    --recode-INFO-all \
    --stdout \
    >chrs_unrelated.vcf

(scan-03-filter_genotypes) 


In [12]:
vcftools \
    --vcf autosomes.vcf \
    --keep relatives_to_keep.txt   \
    --recode \
    --recode-INFO-all \
    --stdout \
    >autosomes_unrelated.vcf

(scan-03-filter_genotypes) 


# Maf and LD filter

## all chromosomes

In [13]:
vcftools \
    --vcf chrs_unrelated.vcf \
    --maf 0.05 \
    --recode \
    --recode-INFO-all \
    --stdout \
    >chrs_unrelated_maf05.vcf

(scan-03-filter_genotypes) 


In [14]:
#ld filtering
plink \
    --vcf chrs_unrelated_maf05.vcf \
    --allow-extra-chr \
    --double-id \
    --indep-pairwise 25 5 0.20 \
    --out chrs_unrelated_maf05_ld_filtered

(scan-03-filter_genotypes) PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to chrs_unrelated_maf05_ld_filtered.log.
Options in effect:
  --allow-extra-chr
  --double-id
  --indep-pairwise 25 5 0.20
  --out chrs_unrelated_maf05_ld_filtered
  --vcf chrs_unrelated_maf05.vcf

1031287 MB RAM detected; reserving 515643 MB for main workspace.
--vcf: chrs_unrelated_maf05_ld_filtered-temporary.bed +
chrs_unrelated_maf05_ld_filtered-temporary.bim +
chrs_unrelated_maf05_ld_filtered-temporary.fam written.
7206957 variants loaded from .bim file.
166 people (0 males, 0 females, 166 ambiguous) loaded from .fam.
Ambiguous sex IDs written to chrs_unrelated_maf05_ld_filtered.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 166 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333

In [15]:
vcftools \
    --vcf chrs_unrelated_maf05.vcf \
    --exclude chrs_unrelated_maf05_ld_filtered.prune.out \
    --recode \
    --recode-INFO-all \
    --stdout \
    >chrs_unrelated_maf05_ld_filtered.vcf

(scan-03-filter_genotypes) 


## autosomes

In [16]:
vcftools \
    --vcf autosomes_unrelated.vcf \
    --maf 0.05 \
    --recode \
    --recode-INFO-all \
    --stdout \
    >autosomes_unrelated_maf05.vcf

(scan-03-filter_genotypes) 


In [17]:
#ld filtering
plink \
    --vcf autosomes_unrelated_maf05.vcf \
    --allow-extra-chr \
    --double-id \
    --indep-pairwise 25 5 0.20 \
    --out autosomes_unrelated_maf05_ld_filtered

(scan-03-filter_genotypes) PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to autosomes_unrelated_maf05_ld_filtered.log.
Options in effect:
  --allow-extra-chr
  --double-id
  --indep-pairwise 25 5 0.20
  --out autosomes_unrelated_maf05_ld_filtered
  --vcf autosomes_unrelated_maf05.vcf

1031287 MB RAM detected; reserving 515643 MB for main workspace.
--vcf: autosomes_unrelated_maf05_ld_filtered-temporary.bed +
autosomes_unrelated_maf05_ld_filtered-temporary.bim +
autosomes_unrelated_maf05_ld_filtered-temporary.fam written.
5864047 variants loaded from .bim file.
166 people (0 males, 0 females, 166 ambiguous) loaded from .fam.
Ambiguous sex IDs written to autosomes_unrelated_maf05_ld_filtered.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 166 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516

In [18]:
vcftools \
    --vcf autosomes_unrelated_maf05.vcf \
    --exclude autosomes_unrelated_maf05_ld_filtered.prune.out \
    --recode \
    --recode-INFO-all \
    --stdout \
    >autosomes_unrelated_maf05_ld_filtered.vcf

(scan-03-filter_genotypes) 
